In [1]:
import numpy as np
import pandas as pd
import glob
import time


import os
import matplotlib.pyplot as plt
import keras
import cv2


In [2]:
df=pd.read_csv('./xss.csv', encoding='utf-8')

In [3]:
df.head()


,Unnamed: 0,kod,label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [4]:
kod=df['kod'].values
kod[1]

'<tt onmouseover="alert(1)">test</tt>'

In [5]:
print(len(kod))


1021


In [6]:
#prebacivanje u ASCII zbog određenih znakova koji imaju velike vrijednosti
#uklanjanje znakova koji imaju veće vrijednosti od 8222
#za ostale vrijednosti veće od 128 i manje od 8222 dodijeljuju se vrijednosti koje se mogu lako normalizirati
def convert_to_ascii(kod):
    kod_ascii=[]
    for i in kod:       
        if(ord(i)<8222):
            if(ord(i)==8217):
                kod_ascii.append(134)      
            if(ord(i)==8221):
                kod_ascii.append(129) 
            if(ord(i)==8220):
                kod_ascii.append(130)   
            if(ord(i)==8216):
                kod_ascii.append(131)   
            if(ord(i)==8217):
                kod_ascii.append(132)
            if(ord(i)==8211):
                kod_ascii.append(133)
            if (ord(i)<=128):
                    kod_ascii.append(ord(i))
            else:
                    pass
                
    zer=np.zeros((10000))
    for i in range(len(kod_ascii)):
        zer[i]=kod_ascii[i]
    zer.shape=(100, 100)
    return zer

In [7]:
#svaki kod u df pretvori u ascii

arr=np.zeros((len(kod),100,100))

for i in range(len(kod)):    
    image=convert_to_ascii(kod[i])
    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128       
    arr[i]=image

In [8]:
print("Oblik : ", arr.shape)


Oblik :  (1021, 100, 100)


In [9]:
#izmjena oblika podataka da bi odgovaralo za CNN  AAAAAAA
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [10]:
data.shape


(1021, 100, 100, 1)

In [11]:
y=df['label'].values


In [12]:
#podijela u test/učenje podatke
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

In [13]:
#import za model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

In [14]:
# CNN model, broj slojeva 11, broj konvolucijskih slojeva 3

model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [15]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0

In [28]:
batch_size = 128
num_epoch = 10
#model
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY))

Epoch 1/10
7/7 [==============================] - 37s 5s/step - loss: 0.3454 - accuracy: 0.8505 - val_loss: 0.4074 - val_accuracy: 0.8146
Epoch 2/10
7/7 [==============================] - 37s 5s/step - loss: 0.3404 - accuracy: 0.8566 - val_loss: 0.4099 - val_accuracy: 0.8195
Epoch 3/10
7/7 [==============================] - 38s 5s/step - loss: 0.3109 - accuracy: 0.8750 - val_loss: 0.3768 - val_accuracy: 0.8390
Epoch 4/10
7/7 [==============================] - 39s 6s/step - loss: 0.2812 - accuracy: 0.8958 - val_loss: 0.3452 - val_accuracy: 0.8829
Epoch 5/10
7/7 [==============================] - 40s 6s/step - loss: 0.2338 - accuracy: 0.9240 - val_loss: 0.3102 - val_accuracy: 0.8927
Epoch 6/10
7/7 [==============================] - 41s 6s/step - loss: 0.1738 - accuracy: 0.9461 - val_loss: 0.3020 - val_accuracy: 0.8634
Epoch 7/10
7/7 [==============================] - 51s 7s/step - loss: 0.1214 - accuracy: 0.9510 - val_loss: 0.2281 - val_accuracy: 0.9317
Epoch 8/10
7/7 [==================

In [29]:
#predikcija za set poadataka za treniranje
pred=model.predict(testX)

In [30]:
# predviđene vrijednosti

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [31]:
#izbroji točne predviđene i krivo predviđene vrijednosti

true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("točno predviđeno :: ", true)
print("krivo predviđeno :: ", false)

točno predviđeno ::  193
krivo predviđeno ::  12


In [32]:
#broj malicioznih i normalnih kodova u setu podataka

mal=0
good=0
for i in range(len(testY)):
    if testY[i]==1:
        mal+=1
    else:
        good+=1

print("Maliciozni :: ", mal)
print("Normalni :: ", good)

Maliciozni ::  110
Normalni ::  95


In [33]:
# definiranje fukcije koja uzima točnu predviđene normalne, točno predviđene maliciozne, krivo predviđene normalne i krivo predviđene maliciozne
# vraća točnost

def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [34]:
# uzima prave pozitivne i krive pozitivne rez
# vraća preciznost

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [35]:
#Uzima točne pozitivne i krive negativne

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [36]:
#Uzima stvarne vrijednosti i predviđene 


def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [37]:
accuracy,precision,recall=confusion_matrix(testY,pred)


In [38]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))


 Accuracy : 0.9414634146341463 
 Precision : 0.9152542372881356 
 Recall : 0.9818181818181818
